# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.core import Pipeline
import json
import pickle
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Getting available workspace
ws = Workspace.from_config()

experiment_name = 'my-automl-project'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

SDK version: 1.20.0
Workspace name: quick-starts-ws-136685
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-136685


In [6]:
## Creating compute cluster
cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
'''
datastore= ws.get_default_datastore()

found = False
key = "Breast Cancer Dataset"
description_text = "Wisconsin Breast Cancer Dataset downloaded from Kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("found dataset")

if not found:
        # Upload csv file into default datastore
        print("uploading csv file")
        datastore.upload_files(files = ['./breast_cancer_dataset.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
        
        # Create AML Dataset and register it into Workspace
        print("creating dataset")
        ###dataset = Dataset.Tabular.from_delimited_files(datastore.path('./breast_cancer_dataset.csv'))        
        dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, './breast_cancer_dataset.csv')])
        
        #Register Dataset in Workspace
        print("registering dataset")
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
        print("created dataset")
'''

data_key = "cancer-dataset"
if data_key in ws.datasets.keys():
    dataset = ws.datasets[data_key]

df = dataset.to_pandas_dataframe()
df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [8]:
## Create scoring dataset by dropping the result column
dataset_score = df.drop('diagnosis', axis=1)
dataset_score.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
project_folder = "./project"

## Setting parameters for AutoMLConfig
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [15]:
## Submitting automl run
automl_experiment = Experiment(ws,'automl_test_experiment')
automl_run = automl_experiment.submit(config=automl_config,
                                     show_output=True)


Running on remote.
No run_configuration provided, running on my-cluster with default configuration
Running on remote compute: my-cluster
Parent Run ID: AutoML_7e9e945c-c414-4dfd-bf9e-03537d11e44f

Current status: FeaturesGeneration. Generating features for the dataset.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
## Use RunDetails widget to show details of the experiment in progress
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == 'Completed')

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [22]:
## Retrieving best automl model
'''
automl_best_run = automl_run.get_best_run_by_primary_metric()
automl_best_run_mertics = automl_best_run.get_metrics()
#parameter_values = automl_best_run.get_details()['runDefinition']['Arguments']
print('best run id : ', automl_best_run.id)
print('best run accuracy :', automl_best_run_metrics['Accuracy'])

'''
automl_best_run, fitted_model = automl_run.get_output()
best_metrics = automl_best_run.get_metrics()
print(best_metrics)


{'precision_score_macro': 0.9869778613199666, 'average_precision_score_micro': 0.9959155160494019, 'f1_score_weighted': 0.9857732420254106, 'norm_macro_recall': 0.966587462082912, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_7e9e945c-c414-4dfd-bf9e-03537d11e44f_53/accuracy_table', 'recall_score_weighted': 0.9858709273182958, 'log_loss': 0.08674221483809967, 'balanced_accuracy': 0.9832937310414561, 'average_precision_score_macro': 0.9960422676082861, 'average_precision_score_weighted': 0.9962556164424937, 'f1_score_macro': 0.9845395761960593, 'AUC_macro': 0.9958241763327115, 'AUC_weighted': 0.9958241763327113, 'matthews_correlation': 0.9701924242935384, 'accuracy': 0.9858709273182958, 'recall_score_macro': 0.9832937310414561, 'precision_score_micro': 0.9858709273182958, 'precision_score_weighted': 0.9867255400929225, 'f1_score_micro': 0.9858709273182958, 'weighted_accuracy': 0.9884072486701427, 'AUC_micro': 0.9958895221292579, 'recall_score_micro': 0.9858709273182958, '

In [25]:

## Saving best automl model
best_run_model_name = 'Udacity-automl-model'
best_run_model_path = 'outputs/model'
model = automl_run.register_model(model_name=best_run_model_name)
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-136685', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-136685'), name=Udacity-automl-model, id=Udacity-automl-model:2, version=2, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [28]:
'''    
env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)
'''
from azureml.core import Model
from azureml.core.webservice import AciWebservice

env = automl_best_run.get_environment()    

inference_config = InferenceConfig(entry_script='score.py',
                                   environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights=True)

service = Model.deploy(ws, "capstone-service", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://71d5f4c0-5d56-459a-a180-b9dde679986e.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_data = json.dumps({
    'data': dataset_score[0:2].to_dict(orient= 'records')
    
})
print(input_data)

In [ ]:
output_data = webservice.run(input_data)
print(output_data)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
## Print logs
print(service.get_logs())

## Delete service
service.delete()